# train with mnist

In [1]:
import numpy as np
import scipy.special

In [2]:
# 신경망 클래스 정의
class MyNeuralNet:
    
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        
        # 입력, 은닉, 출력 계층의 노드 개수 설정
        self.innode = inputnodes
        self.hnode = hiddennodes
        self.outnode = outputnodes
        
        # 학습률
        self.lr = learningrate
        
        # weight
        self.wih = np.random.rand(self.hnode, self.innode) - 0.5 # weight input hidden
        self.who = np.random.rand(self.outnode, self.hnode) - 0.5 # weight hidden output
        
        pass
    
    def printWeight(self):
        print(self.wih.shape, self.wih)
        print(self.who.shape, self.who)
    
    # forward and backward (학습코드는 뒤에서 앞으로)
    def train(self, input_list, target_list):
        input = np.array(input_list, ndmin = 2).T
        
        # input -> hidden
        hidden_input = np.dot(self.wih, input)
        hidden_output = scipy.special.expit(hidden_input)
        
        # hidden -> output
        output_hidden = np.dot(self.who, hidden_output)
        final_outputs = scipy.special.expit(output_hidden)
        
        # Get Error (오차 역전파)
        targets = np.array(target_list, ndmin = 2).T
        output_error = targets - final_outputs # output -> hidden
        hidden_error = np.dot(self.who.T, output_error) # hidden -> input
        
        # Update weight (Gradient descent)
        self.who += self.lr * \
                            np.dot((output_error * final_outputs) * (1.0 - final_outputs), \
                            np.transpose(hidden_output))
        
        self.wih += self.lr * \
                            np.dot((hidden_error * hidden_output) * (1.0 - hidden_output), \
                            np.transpose(input))
        
    # forward
    def inference(self, input_list):
        inputs = np.array(input_list, ndmin = 2).T # 행렬 연산을 위해 1행을 1열로
        
        # input -> hidden
        hidden_input = np.dot(self.wih, inputs) # 행렬 곱연산이므로 순서 주의
        hidden_output = scipy.special.expit(hidden_input) # sigmoid function
        
        # hidden -> output
        output_hidden = np.dot(self.who, hidden_output) 
        final_output = scipy.special.expit(output_hidden)
        
        return final_output

In [3]:
net = MyNeuralNet(4, 4, 3, 0.5)
print(net.inference([10,10,20,20]))
net.train([10, 20, 40, 50], [0.5, 0.2, 0.1])
print(net.inference([10,10,20,20]))

[[0.48517148]
 [0.61187205]
 [0.46834536]]
[[0.48607409]
 [0.58843369]
 [0.44562478]]


# MNIST

In [4]:
!pip install numpy

In [5]:
!mkdir mnist_data

하위 디렉터리 또는 파일 mnist_data이(가) 이미 있습니다.


In [6]:
mnist_train_url = 'http://www.pjreddie.com/media/files/mnist_train.csv'
mnist_test_url = 'http://www.pjreddie.com/media/files/mnist_test.csv'

In [7]:
import shutil
import requests

def download(url, file_name, file_mode='w'):
    response = requests.get(url, stream=True)
    with open(file_name, file_mode) as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response

In [8]:
download(mnist_train_url, 'mnist_data/mnist_train.csv', 'wb')
download(mnist_test_url, 'mnist_data/mnist_test.csv', 'wb')

In [9]:
from tqdm import tqdm # python progressive bar utility => network 지식 필요 (chunk 단위로)
import requests

# url = 'http://www.ovh.net/files/10Mb.dat' # big file test
def download(url, file_name, file_mode = 'wb'):
    # Streaming, so we can iterate over the response
    response = requests.get(url, stream=True)
    total_size_in_bytes=int(response.headers.get('content-length', 0)) # 데이터 전체 사이즈
    block_size=1024 # 1 Kilobyte # 데이터 다운 사이즈 단위 조절
    progress_bar=tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
    with open(file_name, file_mode) as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()
    if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
        print('ERROR, something went wrong')

In [10]:
download(mnist_train_url, 'mnist_data/mnist_train.csv', 'wb')
download(mnist_test_url, 'mnist_data/mnist_test.csv.csv', 'wb')

100%|██████████████████████████████████████████████████████████████████████████████| 110M/110M [00:22<00:00, 4.98MiB/s]
100%|████████████████████████████████████████████████████████████████████████████| 18.3M/18.3M [00:03<00:00, 4.67MiB/s]


# train

In [11]:
input_nodes = 784
hidden_nodes = 256 # 사용자 정의
output_nodes = 10

learning_rate = 0.145 # 사용자 정의

epoch = 5 # 학습을 몇 번 반복할 것인지?

n = MyNeuralNet(input_nodes, hidden_nodes, output_nodes, learning_rate)
# n.printWeight()

In [12]:
training_data_file = open('mnist_data/mnist_train.csv', 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [ ]:
from tqdm import tqdm

for i in range(epoch):
    print("============== {} epoch ==============".format(i+1))
    progress_bar = tqdm(total=len(training_data_list), unit='iB', unit_scale=True)
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        targets = np.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        
        progress_bar.update(1)
    progress_bar.close()

  0%|                                                                               | 11.0/60.0k [00:00<09:08, 109iB/s]

============== 1 epoch ==============


100%|██████████████████████████████████████████████████████████████████████████████| 60.0k/60.0k [03:57<00:00, 253iB/s]
  0%|                                                                               | 29.0/60.0k [00:00<03:30, 284iB/s]

============== 2 epoch ==============


100%|██████████████████████████████████████████████████████████████████████████████| 60.0k/60.0k [03:45<00:00, 266iB/s]
  0%|                                                                               | 29.0/60.0k [00:00<03:33, 282iB/s]

============== 3 epoch ==============


 26%|████████████████████▏                                                         | 15.6k/60.0k [00:58<02:45, 269iB/s]

In [ ]:
# for record in training_data_list:
#     all_values = record.split(',')
#     inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
#     targets = np.zeros(output_nodes) + 0.01
#     targets[int(all_values[0])] = 0.99
#     n.train(inputs, targets)

# Test

In [ ]:
test_data_file = open('mnist_data/mnist_test.csv', 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [ ]:
# 정확도
score = []
for record in test_data_list:
    all_values = record.split(',')
    inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    result = n.inference(inputs)
    if int(all_values[0]) == result.argmax():
        score.append(1)
    else:
        score.append(0)
score_array = np.asarray(score)
accuracy = score_array.sum() / score_array.size
print('performance = ', accuracy)

# Custom Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from PIL import Image, ImageOps

img = image.open('my_number/5_001.png')
img_gray = img.convert('L') # pil image mode 'L': 8bit pixel(0~255)
# black: 255
# white: 0
# 8bit pixel로 변환해줘야 255가 아닌 회색들도 잡아줌

print(img)     
print(img_gray)
# img_gray.show()

img_inverted = ImageOps.invert(img_gray) # rgb 반대로 invert
# img_inverted.show()

pixel_list = np.asarray(img_inverted).flatten().tolist() # 2차원 배열(np.asarray) => 1차원 배열로(flatten)
pixel_list.insert(0, 5)
print(pixel_list)

image_array = np.asfarray(pixel_list[1:]).reshape((28,28))
plt.imshow(image_array, cmap='Greys', interpolation='None')
print('Number : ', pixel_list[0])

# Pickle로 Model 저장

In [ ]:
import pickle

In [ ]:
filename = f'{input_nodes}-{hidden_nodes}-{output_nodes}-{learning_rate}-ep{epoch}-acc{accuracy}.obj'

filehandler = open(filename,"wb")
pickle.dump(net, filehandler)
filehandler.close()

file = open(filename, "rb")
loadedNet = pickle.load(file)
file.close

print(loadedNet)
print(loadedNet.inference([10, 10, 20, 2]))

In [ ]:
loaded_net = MyNeuralNet.load("")
print(loaded_net)